In [ ]:
'''
This module create test train split or cross validation set for different columns filtering and oversampling strategy 
'''

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#ßfrom rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
import math
import logging
import utils_function

In [ ]:
if __name__ == "__main__":
    site ='MCRI'
    year=2013
    datafolder = datafolder
    home_directory = "/home/hoyinchan/code/AKI_CDM_PY/"
    pred_end = 7

In [ ]:
def pre_smote(site, datafolder, stg='stg01', suffix='', returnflag=False, random_state=0):
# site = 'MCRI'
# stg='stg01'
# suffix=''
# returnflag=False
# random_state=None
    year=3000

    print('Running pre_smote on site '+site+":"+str(year)+':'+stg, flush = True)
    data = pd.read_pickle(datafolder+site+'/bt3pos'+suffix+'_'+site+'_'+stg+'_'+str(year)+'.pkl')

    label = data['FLAG'] 
    data = data[data.columns[data.columns!='FLAG']]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(data, label, test_size=0.2, random_state=random_state)
    
    if returnflag:
        return X_train, X_test, y_train, y_test
    
    X_train.to_pickle(datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_nofs_raw'+suffix+'.pkl')
    X_test.to_pickle( datafolder+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_nofs_raw'+suffix+'.pkl')    
    y_train.to_pickle(datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_nofs_raw'+suffix+'.pkl')
    y_test.to_pickle( datafolder+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_nofs_raw'+suffix+'.pkl')
    
    print('Finished pre_smote on site '+site+":"+str(year)+':'+stg, flush = True)    

#    CKD Depreciated
#    X_train_ckdg  = X_train[['EGFR','CKD_group']]
#    X_train = X_train.drop(['EGFR','CKD_group'],axis=1)

#    X_test_ckdg = X_test[['EGFR','CKD_group']]
#    X_test = X_test.drop(['EGFR','CKD_group'],axis=1)    
#    X_train_ckdg.to_pickle(datafolder+site+'/X_train_ckdg_'+site+'_'+str(year)+'_'+stg+'_nofs_raw'+suffix+'.pkl')
#    X_test_ckdg.to_pickle( datafolder+site+'/X_test_ckdg_' +site+'_'+str(year)+'_'+stg+'_nofs_raw'+suffix+'.pkl')    
    

In [ ]:
def smote(site, year, stg):    
    print('Running smote on site '+site+":"+str(year)+':'+stg, flush = True)
    
    X_train = pd.read_pickle(datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_'+'_nofs_raw.pkl')
    y_train = pd.read_pickle(datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_'+'_nofs_raw.pkl')
    
    # impute continuous value for SMOTE
    imp_num = sklearn.impute.SimpleImputer()
    imp_num.fit(X_train.select_dtypes(['Int64', 'Float64']))
    X_train_imp = X_train.reset_index().combine_first(pd.DataFrame(imp_num.transform(X_train.select_dtypes(['int64', 'float64'])), columns=X_train.select_dtypes(['int64', 'float64']).columns)).drop('index',axis=1)
    #cat_fea = [X_train_imp.columns.get_loc(c) for c in list(X_train_imp.select_dtypes('bool').columns)]

    # SMOTE
    sm = SMOTENC(categorical_features=X_train_imp.dtypes == 'bool', k_neighbors=5, n_jobs=23)
    X_res, y_res = sm.fit_resample(X_train_imp, y_train)
    
    #Save tables
    X_res.to_pickle(datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_nofs_sm.pkl')
    y_res.to_pickle(datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_nofs_sm.pkl')

    #Just copy
    X_test = pd.read_pickle(datafolder+site+'/X_test_'+site+'_'+str(year)+'_'+stg+'_nofs_raw.pkl')
    y_test = pd.read_pickle(datafolder+site+'/y_test_'+site+'_'+str(year)+'_'+stg+'_nofs_raw.pkl')
    X_test.to_pickle(       datafolder+site+'/X_test_'+site+'_'+str(year)+'_'+stg+'_nofs_sm.pkl')
    y_test.to_pickle(       datafolder+site+'/y_test_'+site+'_'+str(year)+'_'+stg+'_nofs_sm.pkl')
    
    print('Finished smote on site '+site+":"+str(year)+':'+stg, flush = True)

In [ ]:
def copy_minor(site, year, stg):
    
    print('Running copy_minor on site '+site+":"+str(year)+':'+stg, flush = True)
    X_train = pd.read_pickle(datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_nofs_raw.pkl')
    y_train = pd.read_pickle(datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_nofs_raw.pkl')   
    
    bt_data = X_train
    bt_data.insert(0,'FLAG',y_train)
    
    countsflag = bt_data['FLAG'].value_counts()
    ratio = math.floor(countsflag[0]/countsflag[1])
    data1 = bt_data[bt_data['FLAG']==1]
    data1 = pd.concat([data1]*ratio, axis=0, ignore_index=True)   
    bt_data = pd.concat([data1, bt_data[bt_data['FLAG']==0]], axis=0, ignore_index=True)
    countsflag2 = bt_data['FLAG'].value_counts()
    ratio2 = countsflag2[0] - countsflag2[1]
    data2 = bt_data[bt_data['FLAG']==1].sample(n=ratio2)
    bt_data = pd.concat([data2, bt_data], axis=0, ignore_index=True)        

    y_cp = bt_data['FLAG']
    X_cp = bt_data[bt_data.columns[bt_data.columns!='FLAG']]

    X_cp.to_pickle(datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_nofs_cp.pkl')
    y_cp.to_pickle(datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_nofs_cp.pkl')

    #Just copy
    X_test = pd.read_pickle(datafolder+site+'/X_test_'+site+'_'+str(year)+'_'+stg+'_nofs_raw.pkl')
    y_test = pd.read_pickle(datafolder+site+'/y_test_'+site+'_'+str(year)+'_'+stg+'_nofs_raw.pkl')
    X_test.to_pickle(       datafolder+site+'/X_test_'+site+'_'+str(year)+'_'+stg+'_nofs_cp.pkl')
    y_test.to_pickle(       datafolder+site+'/y_test_'+site+'_'+str(year)+'_'+stg+'_nofs_cp.pkl')    
    
    print('Finished copy_minor on site '+site+":"+str(year)+':'+stg, flush = True)

In [ ]:
def rm_scr_bun(X_train):
    rm_key = ['2160-0','38483-4','14682-9','21232-4','35203-9','44784-7','59826-8',
                  '16188-5','16189-3','59826-8','35591-7','50380-5','50381-3','35592-5',
                  '44784-7','11041-1','51620-3','72271-0','11042-9','51619-5','35203-9','14682-9',
                  '12966-8','12965-0','6299-2','59570-2','12964-3','49071-4','72270-2',
                  '11065-0','3094-0','35234-4','14937-7',
                  '48642-3','48643-1',
                  '3097-3','44734-2']        
    rm_col = [x for x in X_train.columns if 'LAB' in x and x.split(':')[2].split('(')[0] in rm_key]
    return X_train.drop(rm_col, axis=1)

In [ ]:
def rm_lab(X_train):
    rm_col = [x for x in X_train.columns if 'LAB' in x]
    return X_train.drop(rm_col, axis=1)

In [ ]:
def only_med(X_train):
    rm_col = [x for x in X_train.columns if 'MED' not in x]
    return X_train.drop(rm_col, axis=1)

In [ ]:
def all_only_med(site, year, stg, oversample='raw', suffix=''):
    print('Running cp_only_med on site '+site+":"+str(year)+':'+stg, flush = True)        
    
    X_train = pd.read_pickle( datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    X_train = only_med(X_train)
    X_train.to_pickle(        datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_onlymed_'+oversample+suffix+'.pkl')

    X_test = pd.read_pickle(  datafolder+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    X_test = only_med(X_test)
    X_test.to_pickle(         datafolder+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_onlymed_'+oversample+suffix+'.pkl')
        
    #Just copy
    y_train = pd.read_pickle( datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    y_train.to_pickle(        datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_onlymed_'+oversample+suffix+'.pkl')    
    y_test = pd.read_pickle(  datafolder+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    y_test.to_pickle(         datafolder+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_onlymed_'+oversample+suffix+'.pkl')    
    
    print('Finished cp_rm_scr_bun on site '+site+":"+str(year)+':'+stg, flush = True)        

In [ ]:
def all_rm_lab(site, year, stg, oversample='raw', suffix=''):
    print('Running cp_rm_lab on site '+site+":"+str(year)+':'+stg, flush = True)        
    
    X_train = pd.read_pickle( datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    X_train = rm_lab(X_train)
    X_train.to_pickle(        datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_rmlab_'+oversample+suffix+'.pkl')

    X_test = pd.read_pickle(  datafolder+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    X_test = rm_lab(X_test)
    X_test.to_pickle(         datafolder+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_rmlab_'+oversample+suffix+'.pkl')
        
    #Just copy
    y_train = pd.read_pickle( datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    y_train.to_pickle(        datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_rmlab_'+oversample+suffix+'.pkl')    
    y_test = pd.read_pickle(  datafolder+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    y_test.to_pickle(         datafolder+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_rmlab_'+oversample+suffix+'.pkl')    
    
    print('Finished cp_rm_scr_bun on site '+site+":"+str(year)+':'+stg, flush = True)        

In [ ]:
def all_rm_scr_bun(site, year, stg, oversample, suffix=''):
    print('Running cp_rm_scr_bun on site '+site+":"+str(year)+':'+stg, flush = True)        
    
    X_train = pd.read_pickle( datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    X_train = rm_scr_bun(X_train)
    X_train.to_pickle(        datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_rmscrbun_'+oversample+suffix+'.pkl')

    X_test = pd.read_pickle(  datafolder+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    X_test = rm_scr_bun(X_test)
    X_test.to_pickle(         datafolder+site+'/X_test_' +site+'_'+str(year)+'_'+stg+'_rmscrbun_'+oversample+suffix+'.pkl')
        
    #Just copy
    y_train = pd.read_pickle( datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    y_train.to_pickle(        datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_rmscrbun_'+oversample+suffix+'.pkl')    
    y_test = pd.read_pickle(  datafolder+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_nofs_'    +oversample+suffix+'.pkl')
    y_test.to_pickle(         datafolder+site+'/y_test_' +site+'_'+str(year)+'_'+stg+'_rmscrbun_'+oversample+suffix+'.pkl')    
    
    print('Finished cp_rm_scr_bun on site '+site+":"+str(year)+':'+stg, flush = True)        

In [ ]:
def generate_all_pre_catboost(configs_variables):
    '''
    This module create test train split for different columns filtering and oversampling strategy 
    '''

    year = 3000
#    configs_variables = utils_function.read_config(site)
    site, datafolder, home_directory = utils_function.get_commons(configs_variables)
    fs = configs_variables['fs']
    stg = configs_variables['stg']
    oversample = configs_variables['oversample']
    drop_correlation_catboost = configs_variables['drop_correlation_catboost']
    random_state = int(configs_variables['random_state'])
    
    #save raw data
    try:
        if drop_correlation_catboost:
            pre_smote(site, datafolder, stg, 'nc', random_state=random_state)
        else:
            pre_smote(site, datafolder, stg, '', random_state=random_state)            
    except Exception as e:
        logging.basicConfig(filename='preboost.log', filemode='a')    
        print('pre_smote ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n', flush = True)
        logging.error('pre_smote ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n')
        logging.shutdown()       


    #generate copy data
    # if oversample == 'cp':
        # try:
        #     copy_minor(site, year, stg)
        # except Exception as e:
        #     logging.basicConfig(filename='preboost.log', filemode='a')    
        #     print('copy_minor ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n', flush = True)
        #     logging.error('copy_minor ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n')
        #     logging.shutdown()       


    #generate smote
    # if oversample == 'sm':    
        # try:
        # #smote(site, year, stg)    
        # except Exception as e:
        #     logging.basicConfig(filename='preboost.log', filemode='a')    
        #     print('smote ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n', flush = True)
        #     logging.error('smote ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n')
        #     logging.shutdown()           

    #generate rm_scr_bun version
    if fs == 'rmscrbun':
        try:
            all_rm_scr_bun(site, year, stg, 'raw', suffix)
            #smote_rm_scr_bun(site, year)
        except Exception as e:
            logging.basicConfig(filename='preboost.log', filemode='a')    
            print('raw_rm_scr_bun ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n', flush = True)
            logging.error('raw_rm_scr_bun ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n')
            logging.shutdown()       

    #generate rm_lab version
    if fs == 'rmlab':    
        try:
            all_rm_lab(site, year, stg, 'raw', suffix)
            #smote_rm_scr_bun(site, year)
        except Exception as e:
            logging.basicConfig(filename='preboost.log', filemode='a')    
            print('all_rm_lab ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n', flush = True)
            logging.error('all_rm_lab ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n')
            logging.shutdown()       

    #generate only_med version
    if fs == 'onlymed':     
        try:
            all_only_med(site, year, stg, 'raw', suffix)
            #smote_rm_scr_bun(site, year)
        except Exception as e:
            logging.basicConfig(filename='preboost.log', filemode='a')    
            print('all_only_med ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n', flush = True)
            logging.error('all_only_med ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n')
            logging.shutdown()       
        
        
    # try:
    #     all_rm_scr_bun(site, year, stg, 'cp')
    #     #smote_rm_scr_bun(site, year)
    # except Exception as e:
    #     logging.basicConfig(filename='preboost.log', filemode='a')    
    #     print('cp_rm_scr_bun ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n', flush = True)
    #     logging.error('raw_rm_scr_bun ERROR!!!!! '+site+":"+str(year)+'\n+++++++++++++++++\n'+str(e)+'\n-------------------\n')
    #     logging.shutdown()       

In [ ]:
def gen_crossvalidate(configs_variables):
    '''
    This module create cross validation dataset
    '''
    year = 3000
#    configs_variables = utils_function.read_config(site)
    site, datafolder, home_directory = utils_function.get_commons(configs_variables)
    fs = configs_variables['fs']
    stg = configs_variables['stg']
    oversample = configs_variables['oversample']
    drop_correlation_catboost = configs_variables['drop_correlation_catboost']    
    n_splits = int(configs_variables['n_splits'])
    random_state = int(configs_variables['random_state'])
    
    if drop_correlation_catboost:
        suffix = 'nc'
    else:
        suffix = ''   
    
    X_train = pd.read_pickle(datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+'.pkl')
    X_test =  pd.read_pickle(datafolder+site+ '/X_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+'.pkl')
    y_train = pd.read_pickle(datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+'.pkl')
    y_test =  pd.read_pickle(datafolder+site+ '/y_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+'.pkl')           

    X = pd.concat([X_train, X_test])
    y = pd.concat([y_train, y_test])

    skf = sklearn.model_selection.StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    skf.get_n_splits(X, y)

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train_fold = X.iloc[train_index,:]
        X_test_fold = X.iloc[test_index,:]
        y_train_fold = y.iloc[train_index]
        y_test_fold = y.iloc[test_index]
        pd.to_pickle(X_train_fold, datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(i)+'.pkl')
        pd.to_pickle(X_test_fold, datafolder+site+'/X_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(i)+'.pkl')
        pd.to_pickle(y_train_fold, datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(i)+'.pkl')
        pd.to_pickle(y_test_fold, datafolder+site+'/y_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(i)+'.pkl')

In [ ]:
def get_boosttrap(configs_variables, numberbt):
    '''
    This module read and return cross validation dataset
    '''
    year = 3000
    site, datafolder, home_directory = utils_function.get_commons(configs_variables)
    fs = configs_variables['fs']
    stg = configs_variables['stg']
    oversample = configs_variables['oversample']
    drop_correlation_catboost = configs_variables['drop_correlation_catboost']    
    if drop_correlation_catboost:
        suffix = 'nc'
    else:
        suffix = '' 

    X_train = pd.read_pickle(datafolder+site+'/X_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(numberbt)+'.pkl')
    X_test =  pd.read_pickle(datafolder+site+ '/X_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(numberbt)+'.pkl')
    y_train = pd.read_pickle(datafolder+site+'/y_train_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(numberbt)+'.pkl')
    y_test =  pd.read_pickle(datafolder+site+ '/y_test_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+suffix+str(numberbt)+'.pkl')           
    return X_train, X_test, y_train, y_test